# About

This Jupyter notebook contains the beginnings of our analysis code for the [PCAWG](https://dcc.icgc.org/pcawg) infrastructure paper.  This paper describes our efforts to run the core analysis for the project, namely the alignment workflow with 3 variant calling workflows.  We built various infrastructure components that let us do this in a distributed way across many (14) cloud and HPC environments.

The main text of the paper is currently in a private Google document.  This notebook will be used (maybe) to generate our figures if I can figure out how to program in Python!


In [22]:
%matplotlib inline

In [53]:
import matplotlib.pyplot as plt
import numpy as np
import gzip
from io import StringIO
from io import BytesIO
import urllib
import json

# file content
file_content = ""

request = urllib.request.Request('http://pancancer.info/gnos_metadata/latest/donor_p_151215020209.jsonl.gz')
request.add_header('Accept-encoding', 'gzip')
response = urllib.request.urlopen(request)
buf = BytesIO( response.read())
f = gzip.GzipFile(fileobj=buf)
file_content = f.read()

print(file_content)

print("HERE")